# Emitrr Assignment

## Zero-Shot Classification for Sentiment & Intent Classification

In [ ]:
from transformers import pipeline

zero_shot = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


Seeking reassurance


### Sentiment

In [ ]:
text = "You'll be completely alright, no need to worry"
labels = ["Anxious", "Neutral", "Reassured"]
result = zero_shot(text, candidate_labels=labels)
print(result["labels"][0])  # Output: "Seeking reassurance"

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Reassured


### Intent

In [ ]:
text = "I hope it gets better soon."
labels = ["Seeking reassurance", "Reporting symptoms", "Expressing concern"]
result = zero_shot(text, candidate_labels=labels)
print(result["labels"][0])  # Output: "Seeking reassurance"

Expressing concern


In [ ]:
text = "I’m doing better, but I still have some discomfort now and then."
result = zero_shot(text, candidate_labels=labels, multi_label=True)
print(result["labels"][0])
result

Expressing concern


{'sequence': 'I’m doing better, but I still have some discomfort now and then.',
 'labels': ['Expressing concern', 'Reporting symptoms', 'Seeking reassurance'],
 'scores': [0.9805284142494202, 0.9751019477844238, 0.7698802351951599]}

In [ ]:
text = "Out of nowhere, another car hit me from behind, which pushed my car into the one in front."
result = zero_shot(text, candidate_labels=labels, multi_label=True)
print(result["labels"][0])

Expressing concern


In [ ]:
result

{'sequence': 'Out of nowhere, another car hit me from behind, which pushed my car into the one in front.',
 'labels': ['Expressing concern', 'Reporting symptoms', 'Seeking reassurance'],
 'scores': [0.9517746567726135, 0.780878484249115, 0.42864322662353516]}

## NER

### Using Gliner

In [ ]:
from gliner import GLiNER
model = GLiNER.from_pretrained("urchade/gliner_multi")

input_text = """Physician: Good morning, Ms. Jones. How are you feeling today?

Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?

Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?

Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?

Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?

Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?

Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?

Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.

Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?

Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?

Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.

Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.

[Physical Examination Conducted]

Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That’s a relief!

Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.

Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?

Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.

Patient: Thank you, doctor. I appreciate it.

Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""
labels = ["Symptoms", "Treatment", "Diagnosis", "Prognosis"]

entities = model.predict_entities(input_text, labels)
entities_ref = {entity["label"]:[entity["text"]] for entity in entities}
print(entities_ref)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/gliner/data_processing/processor.py:296: UserWarning: Sentence of length 690 has been truncated to 384
  warnings.warn(f"Sentence of length {len(tokens)} has been truncated to {max_len}")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'Symptoms': ['difficulty concentrating'], 'Treatment': ['physiotherapy']}


In [ ]:
for conv in input_text.split("\n"):
  entities = model.predict_entities(conv, labels)
  entities_ref = {entity["label"]:[entity["text"]] for entity in entities}
  print(entities_ref)

{}
{}
{'Symptoms': ['discomfort']}
{}
{}
{}
{}
{}
{}
{}
{'Diagnosis': ['Patient']}
{}
{'Prognosis': ['accident']}
{}
{'Symptoms': ['pain']}
{}
{'Treatment': ['medical attention']}
{}
{'Symptoms': ['whiplash injury'], 'Diagnosis': ['X-rays']}
{}
{}
{}
{'Symptoms': ['discomfort'], 'Treatment': ['physiotherapy']}
{}
{'Symptoms': ['pain']}
{}
{'Symptoms': ['backaches'], 'Prognosis': ['before']}
{}
{'Symptoms': ['difficulty concentrating']}
{}
{'Symptoms': ['emotional issues'], 'Prognosis': ['accident']}
{}
{}
{}
{}
{}
{'Diagnosis': ['physical examination'], 'Symptoms': ['lingering pain']}
{}
{'Treatment': ['Physical Examination']}
{}
{'Symptoms': ['tenderness'], 'Prognosis': ['good condition']}
{}
{'Diagnosis': ['Patient']}
{}
{'Prognosis': ['recovery'], 'Symptoms': ['degeneration']}
{}
{'Prognosis': ['future']}
{}
{'Prognosis': ['full recovery'], 'Symptoms': ['symptoms'], 'Treatment': ['follow-up']}
{}
{}
{}
{}
{}


### Using Pre-Trained BioMedical-NER

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline

# Load a pre-trained biomedical NER model
model_name = "d4data/biomedical-ner-all"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a NER pipeline
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer)

# Test input text
text = "The patient was diagnosed with pneumonia and prescribed amoxicillin."

# Get NER predictions
entities = nlp_ner(text)
print(entities)

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/5.00k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Device set to use cuda:0


[{'entity': 'B-Disease_disorder', 'score': np.float32(0.99975926), 'index': 6, 'word': 'pneumonia', 'start': 31, 'end': 40}, {'entity': 'B-Medication', 'score': np.float32(0.99989533), 'index': 9, 'word': 'am', 'start': 56, 'end': 58}, {'entity': 'B-Medication', 'score': np.float32(0.9982437), 'index': 10, 'word': '##ox', 'start': 58, 'end': 60}, {'entity': 'I-Medication', 'score': np.float32(0.9227398), 'index': 11, 'word': '##ici', 'start': 60, 'end': 63}]


In [ ]:
entities

[{'entity': 'B-Disease_disorder',
  'score': np.float32(0.99975926),
  'index': 6,
  'word': 'pneumonia',
  'start': 31,
  'end': 40},
 {'entity': 'B-Medication',
  'score': np.float32(0.99989533),
  'index': 9,
  'word': 'am',
  'start': 56,
  'end': 58},
 {'entity': 'B-Medication',
  'score': np.float32(0.9982437),
  'index': 10,
  'word': '##ox',
  'start': 58,
  'end': 60},
 {'entity': 'I-Medication',
  'score': np.float32(0.9227398),
  'index': 11,
  'word': '##ici',
  'start': 60,
  'end': 63}]

In [ ]:
# Define custom mapping
entity_mapping = {
    "DISEASE": "Symptom/Diagnosis",
    "MEDICATION": "Treatment",
    "PROCEDURE": "Diagnosis",
    "PROGNOSIS": "Prognosis"
}

# Convert NER results to custom categories
def map_entities(text):
    ner_results = nlp_ner(text)
    mapped_entities = [(ent["word"], entity_mapping.get(ent["entity"], "Other")) for ent in ner_results]
    return mapped_entities

# Test mapping
text = "Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home."
mapped_entities = map_entities(text)
print(mapped_entities)


[]


### Using Pre-Trained Bert on Clinical NER Task

In [ ]:
from transformers import pipeline

pipe = pipeline("token-classification", model="samrawal/bert-base-uncased_clinical-ner")

config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
ner_results = pipe(text)

def reconstruct_entities(ner_results, text):
    entities = []
    current_entity = None

    for entry in ner_results:
        word = entry["word"]
        word_cleaned = word.replace("##", "")  # Remove '##' from subwords
        entity_type = entry["entity"]
        start, end = entry["start"], entry["end"]

        # If it's a B- tag or a new entity type, start a new entity
        if "B-" in entity_type or (current_entity and current_entity["type"] != entity_type):
            if current_entity:
                entities.append(current_entity)  # Save the previous entity

            current_entity = {"word": word_cleaned, "type": entity_type, "start": start, "end": end}

        # If it's an I- tag of the same type, merge it correctly
        elif "I-" in entity_type and current_entity and current_entity["type"][2:] == entity_type[2:]:
            # Check if the word is a subword (it starts with "##" in original form)
            if word.startswith("##"):
                current_entity["word"] += word_cleaned  # Merge without space
            else:
                current_entity["word"] += " " + word_cleaned  # Merge with space

            current_entity["end"] = end  # Update the end position

    # Append the last entity
    if current_entity:
        entities.append(current_entity)

    return entities

# Reconstruct entities
merged_entities = reconstruct_entities(ner_results, text)

# Print results
for entity in merged_entities:
    print(f"Entity: {entity['word']}, Type: {entity['type']}, Start: {entity['start']}, End: {entity['end']}")


Entity: a, Type: B-problem, Start: 86, End: 87
Entity: whiplash injury, Type: I-problem, Start: 88, End: 103
Entity: any, Type: B-test, Start: 124, End: 127
Entity: x - rays, Type: I-test, Start: 128, End: 134


In [ ]:
input_text = """Physician: Good morning, Ms. Jones. How are you feeling today?

Patient: Good morning, doctor. I’m doing better, but I still have some discomfort now and then.

Physician: I understand you were in a car accident last September. Can you walk me through what happened?

Patient: Yes, it was on September 1st, around 12:30 in the afternoon. I was driving from Cheadle Hulme to Manchester when I had to stop in traffic. Out of nowhere, another car hit me from behind, which pushed my car into the one in front.

Physician: That sounds like a strong impact. Were you wearing your seatbelt?

Patient: Yes, I always do.

Physician: What did you feel immediately after the accident?

Patient: At first, I was just shocked. But then I realized I had hit my head on the steering wheel, and I could feel pain in my neck and back almost right away.

Physician: Did you seek medical attention at that time?

Patient: Yes, I went to Moss Bank Accident and Emergency. They checked me over and said it was a whiplash injury, but they didn’t do any X-rays. They just gave me some advice and sent me home.

Physician: How did things progress after that?

Patient: The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort.

Physician: That makes sense. Are you still experiencing pain now?

Patient: It’s not constant, but I do get occasional backaches. It’s nothing like before, though.

Physician: That’s good to hear. Have you noticed any other effects, like anxiety while driving or difficulty concentrating?

Patient: No, nothing like that. I don’t feel nervous driving, and I haven’t had any emotional issues from the accident.

Physician: And how has this impacted your daily life? Work, hobbies, anything like that?

Patient: I had to take a week off work, but after that, I was back to my usual routine. It hasn’t really stopped me from doing anything.

Physician: That’s encouraging. Let’s go ahead and do a physical examination to check your mobility and any lingering pain.

[Physical Examination Conducted]

Physician: Everything looks good. Your neck and back have a full range of movement, and there’s no tenderness or signs of lasting damage. Your muscles and spine seem to be in good condition.

Patient: That’s a relief!

Physician: Yes, your recovery so far has been quite positive. Given your progress, I’d expect you to make a full recovery within six months of the accident. There are no signs of long-term damage or degeneration.

Patient: That’s great to hear. So, I don’t need to worry about this affecting me in the future?

Physician: That’s right. I don’t foresee any long-term impact on your work or daily life. If anything changes or you experience worsening symptoms, you can always come back for a follow-up. But at this point, you’re on track for a full recovery.

Patient: Thank you, doctor. I appreciate it.

Physician: You’re very welcome, Ms. Jones. Take care, and don’t hesitate to reach out if you need anything.
"""


In [ ]:
# Custom label mapping
LABEL_MAPPING = {
    "B-problem": "Symptoms",
    "I-problem": "Symptoms",
    "B-treatment": "Treatment",
    "I-treatment": "Treatment",
    "B-test": "Diagnosis",
    "I-test": "Diagnosis",
    # "O" is ignored
}

OUTPUT = {
  'Symptoms': [],
  'Treatment': [],
  'Diagnosis': []
}


# Function to merge tokens and map categories
def reconstruct_entities(ner_results, text):
    entities = []
    current_entity = None

    for entry in ner_results:
        word = entry["word"].replace("##", "")  # Remove subword markers
        entity_type = entry["entity"]
        start, end = entry["start"], entry["end"]

        # Map the entity to our custom category
        mapped_category = LABEL_MAPPING.get(entity_type, None)

        # If it's a B- tag or a new entity type, start a new entity
        if mapped_category and ("B-" in entity_type or (current_entity and current_entity["type"] != mapped_category)):
            if current_entity:
                entities.append(current_entity)  # Save previous entity

            current_entity = {"word": word, "type": mapped_category, "start": start, "end": end}

        # If it's an I- tag of the same type, merge it correctly
        elif mapped_category and "I-" in entity_type and current_entity and current_entity["type"] == mapped_category:
            if entry["word"].startswith("##"):
                current_entity["word"] += word  # Merge subword (no space)
            else:
                current_entity["word"] += " " + word  # Merge separate word

            current_entity["end"] = end  # Update entity end position

    # Append the last entity
    if current_entity:
        entities.append(current_entity)

    return entities

# Sample text
text = "The first four weeks were rough. My neck and back pain were really bad—I had trouble sleeping and had to take painkillers regularly. It started improving after that, but I had to go through ten sessions of physiotherapy to help with the stiffness and discomfort."

# Get NER predictions
ner_results = pipe(input_text)

# Reconstruct entities with custom categories
mapped_entities = reconstruct_entities(ner_results, text)

# Print results
for entity in mapped_entities:
    if entity["type"] in OUTPUT:
      OUTPUT[entity["type"]].append(entity["word"])

In [ ]:
OUTPUT

{'Symptoms': ['some discomfort',
  'pain in my neck and back',
  'a whiplash injury',
  'my neck and back pain bad',
  'trouble sleeping',
  'the stiffness',
  'discomfort',
  'pain',
  'occasional backaches',
  'other effects',
  'anxiety',
  'difficulty concentrating',
  'nervous',
  'any emotional issues',
  'lingering pain'],
 'Treatment': ['painkillers', 'phys', 'iotherapy'],
 'Diagnosis': ['any x - rays',
  'a physical examination',
  'physical examination']}